# 연습문제 5 - Variational quantum eigensolver


## 역사적 배경

지난 10년간 양자 컴퓨터들은 많은 발전을 이루었고, 파인만의 초기 꿈이었던 자연의 법칙들을 양자 방식으로 시뮬레이션하는 컴퓨팅 시스템에 더 가까워졌습니다. 2014년에 제1저자 Alberto Peruzzo가 발표한 논문은 **Variational Quantum Eigensolver (VQE)**을 소개합니다. 이 알고리즘은 분자의 바닥 상태 에너지 (가장 낮은 에너지)를 찾는데에 쓰이는데, 다른 방법들보다 훨씬 더 낮은 깊이의 회로를 사용합니다.[1] 그리고 2017년에 IBM 퀀텀 팀은 이 VQE 알고리즘을 사용해 수소화리튬 분자의 바닥 상태 에너지를 시뮬레이션하였습니다.[2]

VQE의 마법은 문제의 일부 프로세싱 작업 부하를 고전 컴퓨터에서 아웃소싱하는 것입니다. 이 알고리즘은 ansatz라고 (최적의 가설) 불리는 매개변수화된 양자 회로로 시작하고, 고전 최적화기(optimzer)를 사용해 이 회로에 대한 최적의 매개변수들을 찾아냅니다. 고전 알고리즘보다 VQE가 더 뛰어난 이유는 양자처리장치(QPU)가 문제의 정확한 파동 함수를 나타내고 저장할 수 있기 때문인데, 이는 고전 컴퓨터에서는 지수적으로 어려운 문제입니다. 

이 연습문제 5에서는 여러분이 variational quantum eigensolver를 세팅해 분자의 바닥 상태와 에너지를 알아냄으로써 파인만의 꿈을 직접 깨닫게 될 것입니다. 이는 매우 흥미로운 작업인데, 바닥 상태는 다양한 분자의 성질들을 계산하는 데에 쓰이기 때문입니다. 예를 들어, 핵에 대한 힘의 정확한 크기를 계산할 수 있는데, 이는 시간의 흐름에 따라 화학계에 무슨 일이 일어나는지를 알아내기 위해 분자 동력학 시뮬레이션을 실행하는 데에 쓰입니다.[3]


### 참고 자료

1. Peruzzo, Alberto, et al. "A variational eigenvalue solver on a photonic quantum processor." Nature communications 5.1 (2014): 1-7.
2. Kandala, Abhinav, et al. "Hardware-efficient variational quantum eigensolver for small molecules and quantum magnets." Nature 549.7671 (2017): 242-246.
3. Sokolov, Igor O., et al. "Microcanonical and finite-temperature ab initio molecular dynamics simulations on quantum computers." Physical Review Research 3.1 (2021): 013125.

## 소개

VQE를 실행할 때, 여러분은 시뮬레이션을 어떻게 구성할 것인지 직접 선택할 수 있을 것입니다. 이중에 특히 ansatz 양자 회로의 구성에 대해 집중해서 선택을 하게 될 것입니다.
이는 VQE를 잡음이 많은 양자 컴퓨터에서 실행할 때에 가장 중요한 작업들 중 하나가 바닥 상태를 나타내는 가장 간결한 양자 회로를 찾아 충실도의 손실을 (오차를 초래하는 이유 중 하나입니다) 최소화시키는 것이기 때문입니다.
사실상 이 작업은 결국 정확도를 잃지 않으면서 사용하는 이중 큐비트 게이트의 (예: 제어 반전 게이트 (CNOTs)) 수를 최소화시키는 것입니다.

<div class="alert alert-block alert-success">

<b>목표</b> 

주어진 문제에 대한 바닥 상태를 정확하게 나타내는 가장 짧은 ansatz 회로를 찾아내세요. 창의력을 발휘해 보세요!
    

<b>계획</b> 
    
먼저 가장 작은 분자에 대해 어떻게 VQE 시뮬레이션을 구성하는지 배우고, 이를 더 큰 분자에 대해 적용해 볼 것입니다.
    
**1. 튜토리얼 -  H$_2$에 대한 VQE:** VQE에 익숙해지고 상태 벡터(statevector) 시뮬레이션을 실행하며 최적의 ansatz/고전 최적화기 조합을 찾아보세요.

**2. 최종 도전 과제 - LiH에 대한 VQE** 오로지 상태 벡터 시뮬레이터만을 사용해 첫 번째 부분과 비슷하게 조사하세요. 키스킷의 큐비트 수 축소 책략을 사용해 더 큰 두 번째 시스템에 대해 최적의 회로를 찾아보세요. 회로를 최적화하고, 상상력을 발휘해 매개변수화된 회로에 대해 가장 알맞은 구성 요소들을 찾아내고, 이들을 활용해서 키스킷에서 제공되는 것들보다 더 나은, 바닥 상태에 대한 가장 간결한 ansatz 회로를 구축해 보세요.  

</div>


<div class="alert alert-block alert-danger">

아래는 VQE 시뮬레이션에 관한 이론에 대한 소개입니다. 문제를 풀기 전에 아래의 모든 내용을 이해할 필요는 없습니다. 무서워하지 마세요!

</div>



## 이론

아래의 이미지는 VQE를 사용한 분자 시뮬레이션이 어떻게 양자 컴퓨터에서 시행되는지를 보여주는 대략적인 작업 흐름입니다.

<img src="resources/workflow.png" width=800 height= 1400/>

하이브리도 양자-고전 접근법의 핵심 아이디어는 **중앙처리장치 (CPU)**와 **양자처리장치 (QPU)**에 각각 가장 효율적인 부분들을 아웃소싱하는 것입니다. CPU는 에너지를 계산하기 위해 측정되어야 할 요소들을 열거하고 회로의 매개변수들을 최적화시키는 데에 쓰입니다. QPU는 시스템의 양자 상태를 나타내는 양자 회로를 실행하고 에너지를 측정하는 데에 쓰입니다. 더 자세한 내용은 아래에서 볼 수 있습니다:

**CPU**는 전자 도약과 상호작용에 관련된 에너지들을 효율적으로 계산할 수 있는데 (하트리-폭 계산을 이용한 일체/이체 적분), 이 에너지들은 총 에너지 연산자, 해밀토니안을 나타냅니다. [하트리-폭 (Hartree-Fock, HF) 방법](https://en.wikipedia.org/wiki/Hartree%E2%80%93Fock_method#:~:text=In%20computational%20physics%20and%20chemistry,system%20in%20a%20stationary%20state.)은 후자를 단일 슬레이터 행렬식으로 나타낼 수 있다고 가정함으로써 대략적인 바닥 상태 파동 함수를 효율적으로 계산합니다 (예: 4개의 스핀 궤도와 큐비트를 가진 STO_3G 기저의 H$_2$ 분자에 대해서는, 전자들이 가장 낮은 에너지 스핀 궤도를 차지하는 $|\Psi_{HF} \rangle = |0101 \rangle$입니다). 나중에 VQE에서 QPU가 하는 일은 빠진 전자 연관성과 관련된 다른 상태들을 (예: $i$가 비트 문자열인 $|\Psi \rangle = c_{HF}|\Psi_{HF} \rangle + \sum_i c_i |i\rangle $ 속의 $\sum_i c_i |i\rangle$ 상태들) 나타낼 수 있는 양자 상태를 (상응하는 회로와 그에 대한 매개변수들) 찾는 일입니다. 

HF 계산 다음에, 해밀토니안 속의 연산자들은 페르미온에서 큐비트로의 변환을 (아래의 해밀토니안 부문을 참조하세요) 사용해 QPU의 측정들과 매핑됩니다. 시스템의 성질을 더 분석해 큐비트의 수를 줄이거나 ansatz 회로를 단축시킬 수 있습니다.:

- Z2 대칭성과 2-큐빗 축소에 대해서는 [Bravyi *et al*, 2017](https://arxiv.org/abs/1701.08213v1)를 참고하세요.
- 얽힘 구축에 대해서는 [Eddins *et al.*, 2021](https://arxiv.org/abs/2104.10220v1)를 참고하세요.
- 적응 ansatz에 대해서는 [Grimsley *et al.*,2018](https://arxiv.org/abs/1812.11173v2), [Rattew *et al.*,2019](https://arxiv.org/abs/1910.09694), 그리고 [Tang *et al.*,2019](https://arxiv.org/abs/1911.10205)을 참고하세요. 양자 회로를 단축시킬 방법을 찾을 때 이 문헌들의 아이디어를 참고할 수 있습니다.

**QPU**는 각도 $\vec\theta$로 매개변수화된, 각종 단일 큐비트 회전들과 entanglers(예: 이중 큐비트 게이트)를 활용해 바닥 상태 파동 함수를 나타낼 양자 회로를 실행시킵니다 (아래의 Ansatzes 부문을 참조하세요). 양자 우위는 QPU가 효율적으로 정확한 파동 함수를 나타내고 저장할 수 있다는 데에서 오는데, 고전 컴퓨터에서 이를 몇 개 이상의 원자로 이루어진 시스템에 적용할려면 훨씬 까다롭기 때문입니다. 마지막으로, QPU는 선택한 연산자를 (예: 해밀토니안을 나타내는 연산자) 측정합니다.

아래에서는 VQE 알고리즘의 각 요소에 대해 더 수학적으로 자세하게 설명할 것입니다. 또 이 [VQE에 관한 비디오 에피소드를](https://www.youtube.com/watch?v=Z-A6G0WVI9w) 보면 도움이 될 수도 있습니다.


### 해밀토니안 

여기에서는 주어진 시스템의 에너지를 얻을 때 측정하기 위해 필요한 연산자들을 어떻게 얻을 수 있는지 설명할 것입니다.
이 항들은 다음과 같이 정의되어 있는 분자 해밀토니안에 포함되어 있습니다:
$$
\begin{aligned}
\hat{H} &=\sum_{r s} h_{r s} \hat{a}_{r}^{\dagger} \hat{a}_{s} \\
&+\frac{1}{2} \sum_{p q r s} g_{p q r s} \hat{a}_{p}^{\dagger} \hat{a}_{q}^{\dagger} \hat{a}_{r} \hat{a}_{s}+E_{N N}
\end{aligned}
$$
여기서
$$
h_{p q}=\int \phi_{p}^{*}(r)\left(-\frac{1}{2} \nabla^{2}-\sum_{I} \frac{Z_{I}}{R_{I}-r}\right) \phi_{q}(r)
$$
$$
g_{p q r s}=\int \frac{\phi_{p}^{*}\left(r_{1}\right) \phi_{q}^{*}\left(r_{2}\right) \phi_{r}\left(r_{2}\right) \phi_{s}\left(r_{1}\right)}{\left|r_{1}-r_{2}\right|} 
$$

$h_{r s}$와 $g_{p q r s}$는 (하트리-포크법을 사용한) 일체/이체 적분이고 $E_{N N}$는 핵척력 에너지입니다. 
일체 적분은 전자들의 운동 에너지와 핵과의 상호작용을 나타냅니다. 
이체 적분은 전자들 사이의 상호작용을 나타냅니다.
$\hat{a}_{r}^{\dagger}, \hat{a}_{r}$ 연산자들은 스핀 궤도 $r$에서 전자의 생성과 소멸을 나타내며, 양자 컴퓨터에서 측정되기 위해 연산자들과의 매핑이 필요합니다.
VQE는 전자 에너지를 최소화시키기 때문에 여러분이 핵척력 에너지 $E_{NN}$을 검색하고 더해줘야 총 에너지를 계산할 수 있다는 것에 유의하세요. 
    


따라서 $ h_{r s}$와 $g_{p q r s}$ 텐서들 속의 모든 0이 아닌 행렬 원소에 대해 상응하는 파울리 string(파울리 연산자들의 텐서곱)들을 다음의 페르미온에서 큐비트 변환에 따라 구축할 수 있습니다. 
예를 들어, 궤도 $r = 3$에 대한 요르단-위그너 매핑에서는 다음의 파울리 string을 얻을 수 있습니다:
$$
\hat a_{3}^{\dagger}= \hat \sigma_z \otimes \hat \sigma_z \otimes\left(\frac{ \hat \sigma_x-i \hat \sigma_y}{2}\right) \otimes 1 \otimes \cdots \otimes 1
$$
여기서 $\hat \sigma_x, \hat \sigma_y, \hat \sigma_z$는 잘 알려진 파울리 연산자들입니다. $\hat \sigma_z$ 연산자들의 텐서곱은 페르미온 반교환 관계를 집행하기 위해 사용되었습니다.
아래의 그림은 물 분자의 14개의 스핀 궤도들과 14개의 큐비트들 사이의 요르단-위그너 매핑을 보여줍니다:

<img src="resources/mapping.png" width=600 height= 1200/>


그 다음에는 해밀토니안 속의 일체/이체 들뜸을 (예: $\hat{a}_{r}^{\dagger} \hat{a}_{s}$, $\hat{a}_{p}^{\dagger} \hat{a}_{q}^{\dagger} \hat{a}_{r} \hat{a}_{s}$) 상응하는 파울리 string들과 (예: $\hat{P}_i$, 위의 그림을 참고하세요) 바꿔주기만 하면 됩니다. 결과로 나온 연산자 세트는 QPU에서 측정될 준비가 되었습니다.
추가적인 정보를 위해서는 [Seeley *et al.*, 2012](https://arxiv.org/abs/1208.5986v1)를 참고하세요.

### Ansatzes

화학 문제를 위해 사용할 수 있는 ansatzes들에는 크게 두 종류가 있습니다. 

- **q-UCC ansatzes**들은 물리적으로 영감을 받아, 양자 회로에 전자의 들뜸을 대략적으로 매핑합니다. q-UCCSD ansatz는 (키스킷에서는 `UCCSD`) 가능한 모든 단일과 이중 전자 들뜸을 포함하고 있습니다. 이중 q-pUCCD (`PUCCD`)와 단일 q-UCCD0 (`SUCCD`) 짝은 그저 이러한 모든 들뜸의 부분 집합만을 포함하고 있고 (훨씬 짧은 회로를 의미합니다), 해리 프로파일에 대해 좋은 결과를 제공한다고 증명되었습니다. 예를 들어, q-pUCCD는 단일 들뜸을 포함하고 있지 않고, 이중 들뜸은 아래의 그림처럼 짝지어져 있습니다.
- **Heuristic ansatzes (`TwoLocal`)**들은 회로 깊이를 줄이려고 발명되었으나, 여전히 바닥 상태를 나타낼 수 있습니다. 
아래의 그림과 같이, R 게이트들은 매개변수화된 단일 큐비트 회전들을 나타내며 $U_{CNOT}$ 게이트들은 entanglers(이중 큐비트 게이트)들을 나타냅니다. 여기서의 생각은 똑같은 블럭을 (독립된 매개변수들과 함께) $D$ 번 반복할 경우 바닥 상태에 이를 수 있다는 것입니다. 

추가 정보를 위해서는 [Sokolov *et al.* (q-UCC ansatzes)](https://arxiv.org/abs/1911.10864v2)와 [Barkoutsos *et al.* (Heuristic ansatzes)](https://arxiv.org/pdf/1805.04340.pdf)를 참고해 주세요.

<img src="resources/ansatz.png" width=700 height= 1200/>



### VQE

Given a Hermitian operator $\hat H$ with an unknown minimum eigenvalue $E_{min}$, associated with the eigenstate $|\psi_{min}\rangle$, VQE provides an estimate $E_{\theta}$, bounded by $E_{min}$:

\begin{align*}
    E_{min} \le E_{\theta} \equiv \langle \psi(\theta) |\hat H|\psi(\theta) \rangle
\end{align*}  

where $|\psi(\theta)\rangle$ is the trial state associated with $E_{\theta}$. By applying a parameterized circuit, represented by $U(\theta)$, to some arbitrary starting state $|\psi\rangle$, the algorithm obtains an estimate $U(\theta)|\psi\rangle \equiv |\psi(\theta)\rangle$ on $|\psi_{min}\rangle$. The estimate is iteratively optimized by a classical optimizer by changing the parameter $\theta$ and minimizing the expectation value of $\langle \psi(\theta) |\hat H|\psi(\theta) \rangle$. 

As applications of VQE, there are possibilities in molecular dynamics simulations, see [Sokolov *et al.*, 2021](https://arxiv.org/abs/2008.08144v1), and excited states calculations, see [Ollitrault *et al.*, 2019](https://arxiv.org/abs/1910.12890) to name a few.

<div class="alert alert-block alert-danger">
 
<b> 추가 정보를 위한 참고 자료</b> 

For the qiskit-nature tutorial that implements this algorithm see [here](https://qiskit.org/documentation/nature/tutorials/01_electronic_structure.html)
but this won't be sufficient and you might want to look on the [first page of github repository](https://github.com/Qiskit/qiskit-nature) and the [test folder](https://github.com/Qiskit/qiskit-nature/tree/main/test) containing tests that are written for each component, they provide the base code for the use of each functionality.

</div>

## 첫 번째 부분: 튜토리얼 - H$_2$ 분자에 대한 VQE 



In this part, you will simulate H$_2$ molecule using the STO-3G basis with the PySCF driver and Jordan-Wigner mapping.
We will guide you through the following parts so then you can tackle harder problems.
    


#### 1. Driver

The interfaces to the classical chemistry codes that are available in Qiskit are called drivers.
We have for example `PSI4Driver`, `PyQuanteDriver`, `PySCFDriver` are available. 

By running a driver (Hartree-Fock calculation for a given basis set and molecular geometry), in the cell below, we obtain all the necessary information about our molecule to apply then a quantum algorithm.

In [ ]:
from qiskit_nature.drivers import PySCFDriver

molecule = "H .0 .0 .0; H .0 .0 0.739"
driver = PySCFDriver(atom=molecule)
qmolecule = driver.run()

<div class="alert alert-block alert-danger">
    
<b> Tutorial questions 1</b> 
    
Look into the attributes of `qmolecule` and answer the questions below.

    
1. We need to know the basic characteristics of our molecule. What is the total number of electrons in your system?
2. What is the number of molecular orbitals?
3. What is the number of spin-orbitals?
3. How many qubits would you need to simulate this molecule with Jordan-Wigner mapping?
5. What is the value of the nuclear repulsion energy?

You can find the answers at the end of this notebook.
</div>

In [ ]:
# WRITE YOUR CODE BETWEEN THESE LINES - START




# WRITE YOUR CODE BETWEEN THESE LINES - END

#### 2. Electronic structure problem

You can then create an `ElectronicStructureProblem` that can produce the list of fermionic operators before mapping them to qubits (Pauli strings).

In [ ]:
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
problem = ElectronicStructureProblem(driver)

# Generate the second-quantized operators
second_q_ops = problem.second_q_ops()

# Hamiltonian
main_op = second_q_ops[0]

#### 3. QubitConverter

Allows to define the mapping that you will use in the simulation. You can try different mapping but 
we will stick to `JordanWignerMapper` as allows a simple correspondence: a qubit represents a spin-orbital in the molecule.

In [ ]:
from qiskit_nature.mappers.second_quantization import ParityMapper, BravyiKitaevMapper, JordanWignerMapper
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter

# Setup the mapper and qubit converter
mapper_type = 'JordanWignerMapper'

if mapper_type == 'ParityMapper':
    mapper = ParityMapper()
elif mapper_type == 'JordanWignerMapper':
    mapper = JordanWignerMapper()
elif mapper_type == 'BravyiKitaevMapper':
    mapper = BravyiKitaevMapper()

converter = QubitConverter(mapper=mapper, two_qubit_reduction=False)

# The fermionic operators are mapped to qubit operators
num_particles = (problem.molecule_data_transformed.num_alpha,
             problem.molecule_data_transformed.num_beta)
qubit_op = converter.convert(main_op, num_particles=num_particles)

#### 4. Initial state
As we described in the Theory section, a good initial state in chemistry is the HF state (i.e. $|\Psi_{HF} \rangle = |0101 \rangle$). We can initialize it as follows:

In [ ]:
from qiskit_nature.circuit.library import HartreeFock

num_particles = (problem.molecule_data_transformed.num_alpha,
             problem.molecule_data_transformed.num_beta)
num_spin_orbitals = 2 * problem.molecule_data_transformed.num_molecular_orbitals
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)
print(init_state)

#### 5. Ansatz
One of the most important choices is the quantum circuit that you choose to approximate your ground state.
Here is the example of qiskit circuit library that contains many possibilities for making your own circuit.

In [ ]:
from qiskit.circuit.library import TwoLocal
from qiskit_nature.circuit.library import UCCSD, PUCCD, SUCCD

# Choose the ansatz
ansatz_type = "TwoLocal"

# Parameters for q-UCC antatze
num_particles = (problem.molecule_data_transformed.num_alpha,
             problem.molecule_data_transformed.num_beta)
num_spin_orbitals = 2 * problem.molecule_data_transformed.num_molecular_orbitals

# Put arguments for twolocal
if ansatz_type == "TwoLocal":
    # Single qubit rotations that are placed on all qubits with independent parameters
    rotation_blocks = ['ry', 'rz']
    # Entangling gates
    entanglement_blocks = 'cx'
    # How the qubits are entangled 
    entanglement = 'full'
    # Repetitions of rotation_blocks + entanglement_blocks with independent parameters
    repetitions = 3
    # Skip the final rotation_blocks layer
    skip_final_rotation_layer = True
    ansatz = TwoLocal(qubit_op.num_qubits, rotation_blocks, entanglement_blocks, reps=repetitions, 
                      entanglement=entanglement, skip_final_rotation_layer=skip_final_rotation_layer)
    # Add the initial state
    ansatz.compose(init_state, front=True, inplace=True)
elif ansatz_type == "UCCSD":
    ansatz = UCCSD(converter,num_particles,num_spin_orbitals,initial_state = init_state)
elif ansatz_type == "PUCCD":
    ansatz = PUCCD(converter,num_particles,num_spin_orbitals,initial_state = init_state)
elif ansatz_type == "SUCCD":
    ansatz = SUCCD(converter,num_particles,num_spin_orbitals,initial_state = init_state)
elif ansatz_type == "Custom":
    # Example of how to write your own circuit
    from qiskit.circuit import Parameter, QuantumCircuit, QuantumRegister
    # Define the variational parameter
    theta = Parameter('a')
    n = qubit_op.num_qubits
    # Make an empty quantum circuit
    qc = QuantumCircuit(qubit_op.num_qubits)
    qubit_label = 0
    # Place a Hadamard gate
    qc.h(qubit_label)
    # Place a CNOT ladder
    for i in range(n-1):
        qc.cx(i, i+1)
    # Visual separator
    qc.barrier()
    # rz rotations on all qubits
    qc.rz(theta, range(n))
    ansatz = qc
    ansatz.compose(init_state, front=True, inplace=True)

print(ansatz)

#### 6. Backend
This is where you specify the simulator or device where you want to run your algorithm.
We will focus on the `statevector_simulator` in this challenge.


In [ ]:
from qiskit import Aer
backend = Aer.get_backend('statevector_simulator')

#### 7. Optimizer

The optimizer guides the evolution of the parameters of the ansatz so it is very important to investigate the energy convergence as it would define the number of measurements that have to be performed on the QPU.
A clever choice might reduce drastically the number of needed energy evaluations.

In [ ]:
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP

optimizer_type = 'COBYLA'

# You may want to tune the parameters 
# of each optimizer, here the defaults are used
if optimizer_type == 'COBYLA':
    optimizer = COBYLA(maxiter=500)
elif optimizer_type == 'L_BFGS_B':
    optimizer = L_BFGS_B(maxfun=500)
elif optimizer_type == 'SPSA':
    optimizer = SPSA(maxiter=500)
elif optimizer_type == 'SLSQP':
    optimizer = SLSQP(maxiter=500)

#### 8. Exact eigensolver
For learning purposes, we can solve the problem exactly with the exact diagonalization of the Hamiltonian matrix so we know where to aim with VQE.
Of course, the dimensions of this matrix scale exponentially in the number of molecular orbitals so you can try doing this for a large molecule of your choice and see how slow this becomes. 
For very large systems you would run out of memory trying to store their wavefunctions.

In [ ]:
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
import numpy as np 

def exact_diagonalizer(problem, converter):
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

result_exact = exact_diagonalizer(problem, converter)
exact_energy = np.real(result_exact.eigenenergies[0])
print("Exact electronic energy", exact_energy)
print(result_exact)

# The targeted electronic energy for H2 is -1.85336 Ha
# Check with your VQE result.

#### 9. VQE and initial parameters for the ansatz
Now we can import the VQE class and run the algorithm.

In [ ]:
from qiskit.algorithms import VQE
from IPython.display import display, clear_output

# Print and save the data in lists
def callback(eval_count, parameters, mean, std):  
    # Overwrites the same line when printing
    display("Evaluation: {}, Energy: {}, Std: {}".format(eval_count, mean, std))
    clear_output(wait=True)
    counts.append(eval_count)
    values.append(mean)
    params.append(parameters)
    deviation.append(std)

counts = []
values = []
params = []
deviation = []

# Set initial parameters of the ansatz
# We choose a fixed small displacement 
# So all participants start from similar starting point
try:
    initial_point = [0.01] * len(ansatz.ordered_parameters)
except:
    initial_point = [0.01] * ansatz.num_parameters

algorithm = VQE(ansatz,
                optimizer=optimizer,
                quantum_instance=backend,
                callback=callback,
                initial_point=initial_point)

result = algorithm.compute_minimum_eigenvalue(qubit_op)

print(result)

#### 9. Scoring function 
We need to judge how good are your VQE simulations, your choice of ansatz/optimizer.
For this, we implemented the following simple scoring function:

$$ score = N_{CNOT}$$

where $N_{CNOT}$ is the number of CNOTs. 
But you have to reach the chemical accuracy which is $\delta E_{chem} = 0.004$ Ha $= 4$ mHa, which may be hard to reach depending on the problem. 
You have to reach the accuracy we set in a minimal number of CNOTs to win the challenge. 
The lower the score the better!

In [ ]:
# Store results in a dictionary
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller

# Unroller transpile your circuit into CNOTs and U gates
pass_ = Unroller(['u', 'cx'])
pm = PassManager(pass_)
ansatz_tp = pm.run(ansatz)
cnots = ansatz_tp.count_ops()['cx']
score = cnots

accuracy_threshold = 4.0 # in mHa
energy = result.optimal_value

if ansatz_type == "TwoLocal":
    result_dict = {
        'optimizer': optimizer.__class__.__name__,
        'mapping': converter.mapper.__class__.__name__,
        'ansatz': ansatz.__class__.__name__,
        'rotation blocks': rotation_blocks,
        'entanglement_blocks': entanglement_blocks,
        'entanglement': entanglement,
        'repetitions': repetitions,
        'skip_final_rotation_layer': skip_final_rotation_layer,
        'energy (Ha)': energy,
        'error (mHa)': (energy-exact_energy)*1000,
        'pass': (energy-exact_energy)*1000 <= accuracy_threshold,
        '# of parameters': len(result.optimal_point),
        'final parameters': result.optimal_point,
        '# of evaluations': result.optimizer_evals,
        'optimizer time': result.optimizer_time,
        '# of qubits': int(qubit_op.num_qubits),
        '# of CNOTs': cnots,
        'score': score}
else:
    result_dict = {
        'optimizer': optimizer.__class__.__name__,
        'mapping': converter.mapper.__class__.__name__,
        'ansatz': ansatz.__class__.__name__,
        'rotation blocks': None,
        'entanglement_blocks': None,
        'entanglement': None,
        'repetitions': None,
        'skip_final_rotation_layer': None,
        'energy (Ha)': energy,
        'error (mHa)': (energy-exact_energy)*1000,
        'pass': (energy-exact_energy)*1000 <= accuracy_thresholdaccuract_thres,
        '# of parameters': len(result.optimal_point),
        'final parameters': result.optimal_point,
        '# of evaluations': result.optimizer_evals,
        'optimizer time': result.optimizer_time,
        '# of qubits': int(qubit_op.num_qubits),
        '# of CNOTs': cnots,
        'score': score}

# Plot the results
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1)
ax.set_xlabel('Iterations')
ax.set_ylabel('Energy')
ax.grid()
fig.text(0.7, 0.75, f'Energy: {result.optimal_value:.3f}\nScore: {score:.0f}')
plt.title(f"{result_dict['optimizer']}-{result_dict['mapping']}\n{result_dict['ansatz']}")
ax.plot(counts, values)
ax.axhline(exact_energy, linestyle='--')
fig_title = f"\
{result_dict['optimizer']}-\
{result_dict['mapping']}-\
{result_dict['ansatz']}-\
Energy({result_dict['energy (Ha)']:.3f})-\
Score({result_dict['score']:.0f})\
.png"
fig.savefig(fig_title, dpi=300)

# Display and save the data
import pandas as pd
import os.path
filename = 'results_h2.csv'
if os.path.isfile(filename):
    result_df = pd.read_csv(filename)
    result_df = result_df.append([result_dict])
else:
    result_df = pd.DataFrame.from_dict([result_dict])
result_df.to_csv(filename)
result_df[['optimizer','ansatz', '# of qubits', '# of parameters','rotation blocks', 'entanglement_blocks',
    'entanglement', 'repetitions', 'error (mHa)', 'pass', 'score']]

<div class="alert alert-block alert-danger">
    
<b>Tutorial questions 2</b> 

Experiment with all the parameters and then:

1. Can you find your best (best score) heuristic ansatz (by modifying parameters of `TwoLocal` ansatz) and optimizer?
2. Can you find your best q-UCC ansatz (choose among `UCCSD, PUCCD or SUCCD` ansatzes) and optimizer?
3. In the cell where we define the ansatz, 
   can you modify the `Custom` ansatz by placing gates yourself to write a better circuit than your `TwoLocal` circuit? 

For each question, give `ansatz` objects.
Remember, you have to reach the chemical accuracy $|E_{exact} - E_{VQE}| \leq 0.004 $ Ha $= 4$ mHa.
    
</div>



In [ ]:
# WRITE YOUR CODE BETWEEN THESE LINES - START




# WRITE YOUR CODE BETWEEN THESE LINES - END

## Part 2: Final Challenge - VQE for LiH molecule 


In this part, you will simulate LiH molecule using the STO-3G basis with the PySCF driver.

</div>
    
<div class="alert alert-block alert-success">

<b>Goal</b> 

Experiment with all the parameters and then find your best ansatz. You can be as creative as you want!

For each question, give `ansatz` objects as for Part 1. Your final score will be based only on Part 2.
    
</div>

Be aware that the system is larger now. Work out how many qubits you would need for this system by retrieving the number of spin-orbitals. 

### Reducing the problem size

You might want to reduce the number of qubits for your simulation:
- you could freeze the core electrons that do not contribute significantly to chemistry and consider only the valence electrons. Qiskit  already has this functionality implemented. So inspect the different transformers in `qiskit_nature.transformers`  and find the one that performs the freeze core approximation.
- you could use `ParityMapper` with `two_qubit_reduction=True` to eliminate 2 qubits.
- you could reduce the number of qubits by inspecting the symmetries of your Hamiltonian. Find a way to use `Z2Symmetries` in Qiskit.

### Custom ansatz 

You might want to explore the ideas proposed in [Grimsley *et al.*,2018](https://arxiv.org/abs/1812.11173v2), [H. L. Tang *et al.*,2019](https://arxiv.org/abs/1911.10205), [Rattew *et al.*,2019](https://arxiv.org/abs/1910.09694), [Tang *et al.*,2019](https://arxiv.org/abs/1911.10205). 
You can even get try machine learning algorithms to generate best ansatz circuits.

### Setup the simulation

Let's now run the Hartree-Fock calculation and the rest is up to you!

<div class="alert alert-block alert-danger">

<b>Attention</b> 

We give below the `driver`, the `initial_point`, the `initial_state` that should remain as given.
You are free then to explore all other things available in Qiskit.
So you have to start from this initial point (all parameters set to 0.01):
    
`initial_point = [0.01] * len(ansatz.ordered_parameters)`
    or
`initial_point = [0.01] * ansatz.num_parameters`

and your initial state has to be the Hartree-Fock state:
    
`init_state = HartreeFock(num_spin_orbitals, num_particles, converter)`
    
For each question, give `ansatz` object.
Remember you have to reach the chemical accuracy $|E_{exact} - E_{VQE}| \leq 0.004 $ Ha $= 4$ mHa.

</div>

In [ ]:
from qiskit_nature.drivers import PySCFDriver

molecule = 'Li 0.0 0.0 0.0; H 0.0 0.0 1.5474'
driver = PySCFDriver(atom=molecule)
qmolecule = driver.run()

In [ ]:
# WRITE YOUR CODE BETWEEN THESE LINES - START





# WRITE YOUR CODE BETWEEN THESE LINES - END

In [ ]:
# Check your answer using following code
from qc_grader import grade_ex5
freeze_core = False # change to True if you freezed core electrons
grade_ex5(ansatz,qubit_op,result,freeze_core)

In [ ]:
# Submit your answer. You can re-submit at any time.
from qc_grader import submit_ex5
submit_ex5(ansatz,qubit_op,result,freeze_core)

## Answers for Part 1

<div class="alert alert-block alert-danger">

<b>Questions</b> 
    
Look into the attributes of `qmolecule` and answer the questions below.

    
1. We need to know the basic characteristics of our molecule. What is the total number of electrons in your system?
2. What is the number of molecular orbitals?
3. What is the number of spin-orbitals?
3. How many qubits would you need to simulate this molecule with Jordan-Wigner mapping?
5. What is the value of the nuclear repulsion energy?
    
</div>

<div class="alert alert-block alert-success">

<b>Answers </b> 

1. `n_el = qmolecule.num_alpha + qmolecule.num_beta`
    
2. `n_mo = qmolecule.num_molecular_orbitals`
    
3. `n_so = 2 * qmolecule.num_molecular_orbitals`
    
4. `n_q = 2* qmolecule.num_molecular_orbitals`
    
5. `e_nn = qmolecule.nuclear_repulsion_energy`
    
    
</div>

## Additional information

**Created by:** Igor Sokolov, Junye Huang, Rahul Pratap Singh

**Version:** 1.0.0